# Structured Data Quality Validation

This notebook validates the quality of structured data in our BDM project using Great Expectations.


In [1]:
# Install Great Expectations if needed
# !pip install great-expectations

import great_expectations as ge
import pandas as pd
import duckdb
import os
from pathlib import Path

print("Great Expectations version:", ge.__version__)
print("Setup complete ✓")

Great Expectations version: 1.5.0
Setup complete ✓


In [2]:

# Initialize Great Expectations context
context = ge.get_context(mode="ephemeral")
print(f"Context type: {type(context).__name__}")
print("Great Expectations initialized ✓")

Context type: EphemeralDataContext
Great Expectations initialized ✓


In [3]:
def load_from_duckdb():
    """Load structured data from DuckDB trusted zone"""
    datasets = {}
    
    try:
        con = duckdb.connect("../bdm_project.duckdb") 
        
        # Show available tables
        tables = con.execute("SHOW TABLES").fetchall()
        table_names = [table[0] for table in tables]
        print(f"Available tables: {table_names}")
        
        table_mappings = {
            'environmental': 'trusted_environmental_indicators',
            'passenger': 'trusted_passenger_volume',             
            'administrative': 'trusted_admin_boundaries'         
        }
        
        for dataset_key, table_name in table_mappings.items():
            datasets[dataset_key] = None
            if table_name in table_names:
                try:
                    df = con.execute(f"SELECT * FROM {table_name}").df()
                    datasets[dataset_key] = df
                    print(f"✓ {dataset_key.title()}: {len(df)} rows from {table_name}")
                except Exception as e:
                    print(f"Error loading {table_name}: {e}")
            else:
                print(f"{dataset_key.title()}: Table {table_name} not found")
        
        con.close()
        return datasets
    
    except Exception as e:
        print(f"Error connecting to DuckDB: {e}")
        return {'environmental': None, 'passenger': None, 'administrative': None}

In [4]:
def load_from_files():
    """Fallback: Load data from Parquet files"""
    datasets = {}
    
    file_paths = {
        'environmental': [
            "../storage/delta/trusted/structured/environmental_indicators/",
            "../storage/delta/raw/environmental_indicators/"
        ],
        'passenger': [
            "../storage/delta/trusted/structured/passenger_volume/",
            "../storage/delta/raw/passenger_volume/"
        ],
        'administrative': [
            "../storage/delta/trusted/structured/administrative_shapefiles/",
            "../storage/delta/raw/administrative_shapefiles/"
        ]
    }
    
    for dataset_key, paths in file_paths.items():
        datasets[dataset_key] = None
        
        for path in paths:
            if os.path.exists(path):
                try:
                    # Find parquet files
                    parquet_files = list(Path(path).glob("*.parquet"))
                    if parquet_files:
                        # Get the most recent file
                        latest_file = max(parquet_files, key=os.path.getmtime)
                        df = pd.read_parquet(latest_file)
                        datasets[dataset_key] = df
                        print(f"✓ {dataset_key.title()}: {len(df)} rows from {latest_file.name}")
                        break
                except Exception as e:
                    print(f"Could not load from {path}: {e}")
                    continue
        
        if datasets[dataset_key] is None:
            print(f"{dataset_key.title()}: No files found")
    
    return datasets

In [5]:
print("Loading structured datasets...")
print("=" * 50)

# Try DuckDB first
datasets = load_from_duckdb()

# If no data found, try files
if all(v is None for v in datasets.values()):
    print("\nNo data in DuckDB, trying files...")
    datasets = load_from_files()

print("\nData loading complete!")
print(f"Datasets available: {[k for k, v in datasets.items() if v is not None]}")

Loading structured datasets...
Available tables: ['trusted_admin_boundaries', 'trusted_environmental_indicators', 'trusted_passenger_volume']
✓ Environmental: 7 rows from trusted_environmental_indicators
✓ Passenger: 528 rows from trusted_passenger_volume
✓ Administrative: 10 rows from trusted_admin_boundaries

Data loading complete!
Datasets available: ['environmental', 'passenger', 'administrative']


In [6]:
# Show basic info about each dataset
for name, df in datasets.items():
    if df is not None:
        print(f"\n{name.upper()} DATASET:")
        print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
        print(f"Columns: {list(df.columns)}")
        print(f"Sample data:\n{df.head(2)}")
        print("-" * 50)


ENVIRONMENTAL DATASET:
Rows: 7, Columns: 12
Columns: ['year', 'gwh_raccio_elecrica', 'milloes_liros_diesel_cosumidos', 'gwh_l_diesel', 'gwh_oal', 'iesidad_eergeica_wh_u', 'iesidad_carboo_g_co2_u', 'gasos_iversioes_ambieales_euros', 'cosumo_agua_m3', 'geeracio_residuos_peligrosos_oeladas', 'porceaje_rafico_viajeros_rees_baja_emisio_acusica', 'porceaje_rafico_mercacias_rees_baja_emisio_acusica']
Sample data:
   year  gwh_raccio_elecrica  milloes_liros_diesel_cosumidos  gwh_l_diesel  \
0  2019              2460.30                           72.12         711.8   
1  2018              2388.12                           75.20         742.8   

   gwh_oal  iesidad_eergeica_wh_u  iesidad_carboo_g_co2_u  \
0   3172.1                   94.8                    5.54   
1   3130.9                   94.2                   21.55   

   gasos_iversioes_ambieales_euros  cosumo_agua_m3  \
0                             2755             903   
1                             2538          917605   

   geer

In [7]:
def validate_dataset(df, dataset_name, expectations_list):
    """Run Great Expectations validation on a dataset"""
    
    if df is None:
        print(f"SKIP {dataset_name}: No data available")
        return 0, 0
    
    # Setup Great Expectations objects
    source_name = f"{dataset_name}_source"
    data_source = context.data_sources.add_pandas(name=source_name)
    data_asset = data_source.add_dataframe_asset(name=f"{dataset_name}_data")
    batch_definition = data_asset.add_batch_definition_whole_dataframe(f"{dataset_name}_batch")
    batch = batch_definition.get_batch(batch_parameters={"dataframe": df})
    
    # Run validations
    passed = 0
    total = len(expectations_list)
    
    print(f"\n{dataset_name.upper()} VALIDATION:")
    print("-" * 40)
    
    for i, (description, expectation) in enumerate(expectations_list, 1):
        try:
            result = batch.validate(expectation)
            if result['success']:
                print(f"PASS {i}. {description}")
                passed += 1
            else:
                print(f"FAIL {i}. {description}")
                if 'result' in result:
                    unexpected = result['result'].get('unexpected_count', 0)
                    total_count = result['result'].get('element_count', 0)
                    if total_count > 0:
                        print(f"     {unexpected}/{total_count} values failed")
        except Exception as e:
            print(f"ERROR {i}. {description}")
    
    # Summary
    percentage = (passed / total * 100) if total > 0 else 0
    print(f"\nSUMMARY: {passed}/{total} tests passed ({percentage:.0f}%)")
    
    return passed, total

In [8]:
environmental_expectations = [
    ("Year not null", ge.expectations.ExpectColumnValuesToNotBeNull(column="year")),
    ("Year range 2000-2030", ge.expectations.ExpectColumnValuesToBeBetween(column="year", min_value=2000, max_value=2030))
]

# Run validation
env_passed, env_total = validate_dataset(datasets['environmental'], 'Environmental Indicators', environmental_expectations)



ENVIRONMENTAL INDICATORS VALIDATION:
----------------------------------------


Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 1668.71it/s]


PASS 1. Year not null


Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 2324.49it/s]

PASS 2. Year range 2000-2030

SUMMARY: 2/2 tests passed (100%)


In [ ]:
def get_passenger_expectations(df):
    if df is None:
        return []
    
    columns = df.columns.tolist()
    expectations = []
    
    # Station validation
    station_cols = [col for col in columns if any(keyword in col.lower() for keyword in ['station', 'estacion', 'codigo'])]
    if station_cols:
        expectations.append(("Station code not null", ge.expectations.ExpectColumnValuesToNotBeNull(column=station_cols[0])))
    
    # Passenger count validation
    count_cols = [col for col in columns if any(keyword in col.lower() for keyword in ['subidas', 'bajadas', 'passengers'])]
    for col in count_cols:
        expectations.append((f"{col} non-negative", ge.expectations.ExpectColumnValuesToBeBetween(column=col, min_value=0, max_value=50000)))
    
    return expectations

# Run validation
passenger_expectations = get_passenger_expectations(datasets['passenger'])
pass_passed, pass_total = validate_dataset(datasets['passenger'], 'Passenger Volume', passenger_expectations)


PASSENGER VOLUME VALIDATION:
----------------------------------------


Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 2154.10it/s]

PASS 1. Station code not null

SUMMARY: 1/1 tests passed (100%)


In [10]:
# Get admin expectations based on available columns
def get_admin_expectations(df):
    if df is None:
        return []
    
    columns = df.columns.tolist()
    expectations = []
    
    if 'geometry_wkt' in columns:
        expectations.extend([
            ("Geometry WKT not null", ge.expectations.ExpectColumnValuesToNotBeNull(column="geometry_wkt")),
            ("Geometry WKT is string", ge.expectations.ExpectColumnValuesToBeOfType(column="geometry_wkt", type_="str"))
        ])
    
    # Name validation
    name_cols = [col for col in columns if any(keyword in col.lower() for keyword in ['name', 'nom', 'district'])]
    if name_cols:
        expectations.append(("Name not null", ge.expectations.ExpectColumnValuesToNotBeNull(column=name_cols[0])))
    
    return expectations

# Run validation
admin_expectations = get_admin_expectations(datasets['administrative'])
admin_passed, admin_total = validate_dataset(datasets['administrative'], 'Administrative Boundaries', admin_expectations)


ADMINISTRATIVE BOUNDARIES VALIDATION:
----------------------------------------


Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 2034.96it/s]


PASS 1. Geometry WKT not null


Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 2167.82it/s]


PASS 2. Geometry WKT is string


Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 1618.02it/s]

PASS 3. Name not null

SUMMARY: 3/3 tests passed (100%)


In [11]:
print("\n" + "=" * 50)
print("STRUCTURED DATA QUALITY SUMMARY")
print("=" * 50)

# Calculate totals
total_passed = env_passed + pass_passed + admin_passed
total_tests = env_total + pass_total + admin_total

if total_tests > 0:
    overall_percentage = (total_passed / total_tests * 100)
    
    print(f"OVERALL: {total_passed}/{total_tests} tests passed ({overall_percentage:.0f}%)")
    
    # Individual summaries
    if env_total > 0:
        print(f"Environmental: {env_passed}/{env_total} ({env_passed/env_total*100:.0f}%)")
    if pass_total > 0:
        print(f"Passenger: {pass_passed}/{pass_total} ({pass_passed/pass_total*100:.0f}%)")
    if admin_total > 0:
        print(f"Administrative: {admin_passed}/{admin_total} ({admin_passed/admin_total*100:.0f}%)")
    
    # Quality assessment
    if overall_percentage >= 90:
        print("\nSTATUS: Excellent data quality")
    elif overall_percentage >= 75:
        print("\nSTATUS: Good data quality")
    elif overall_percentage >= 50:
        print("\nSTATUS: Acceptable data quality")
    else:
        print("\nSTATUS: Poor data quality")
else:
    print("No datasets available for validation")

print("=" * 50)
print("Validation Complete")


STRUCTURED DATA QUALITY SUMMARY
OVERALL: 6/6 tests passed (100%)
Environmental: 2/2 (100%)
Passenger: 1/1 (100%)
Administrative: 3/3 (100%)

STATUS: Excellent data quality
Validation Complete
